In [3]:
import boto3

s3_bucket = "sagemaker-us-east-1-198739141498"  # Change this
s3_prefix = "deberta-finetuned-model/"  # Change this

s3_client = boto3.client("s3")
response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix=s3_prefix)

if "Contents" in response:
    print("Files found in S3:")
    for obj in response["Contents"]:
        print(obj["Key"])
else:
    print("❌ No files found! Check your S3 path.")


Files found in S3:
deberta-finetuned-model/added_tokens.json
deberta-finetuned-model/config.json
deberta-finetuned-model/model.safetensors
deberta-finetuned-model/model.tar.gz
deberta-finetuned-model/special_tokens_map.json
deberta-finetuned-model/spm.model
deberta-finetuned-model/tokenizer.json
deberta-finetuned-model/tokenizer_config.json
deberta-finetuned-model/training_args.bin


In [3]:
import tarfile
import os

model_dir = "trained_model"  # Folder where your trained model is stored
output_filename = "model.tar.gz"

with tarfile.open(output_filename, "w:gz") as tar:
    tar.add(model_dir, arcname=".")

print(f"✅ Created {output_filename}")

✅ Created model.tar.gz


In [4]:
import boto3

s3_bucket = "sagemaker-us-east-1-198739141498"  # Change to your actual bucket
s3_key = "deberta-finetuned-model/model.tar.gz"

# s3_client = boto3.client("s3")
# s3_client.upload_file("model.tar.gz", s3_bucket, s3_key)

print(f"✅ Uploaded model.tar.gz to s3://{s3_bucket}/{s3_key}")


✅ Uploaded model.tar.gz to s3://sagemaker-us-east-1-198739141498/deberta-finetuned-model/model.tar.gz


In [5]:
response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix="deberta-finetuned-model/")
if "Contents" in response:
    print("Files found in S3:")
    for obj in response["Contents"]:
        print(obj["Key"])
else:
    print("❌ No files found! Check your S3 path.")

Files found in S3:
deberta-finetuned-model/added_tokens.json
deberta-finetuned-model/config.json
deberta-finetuned-model/model.safetensors
deberta-finetuned-model/model.tar.gz
deberta-finetuned-model/special_tokens_map.json
deberta-finetuned-model/spm.model
deberta-finetuned-model/tokenizer.json
deberta-finetuned-model/tokenizer_config.json
deberta-finetuned-model/training_args.bin


In [17]:
import boto3
import sagemaker
# Define your S3 bucket name
s3_bucket = "sagemaker-us-east-1-198739141498"
s3_model_path = f"s3://{s3_bucket}/deberta-finetuned-model/model.tar.gz"
# Define IAM role
role = "arn:aws:iam::198739141498:role/ClaimCat-SageMaker-Execution-Role"
# Define model name
model_name = "deberta-finetuned-model"
# Create SageMaker session
sagemaker_session = sagemaker.Session()
# Create the SageMaker PyTorch model

from sagemaker.pytorch import PyTorchModel

pytorch_model = PyTorchModel(
    model_data=s3_model_path,
    role=role,
    entry_point="inference.py",
    source_dir="./model_inf", # Directory containing inference.py
    framework_version="1.12.1",
    py_version="py38",
    predictor_cls=sagemaker.predictor.Predictor
)
# Deploy the model with error handling
try:
    predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large"
    )
    print("Model deployed successfully!")
    print(f"Endpoint Name: {predictor.endpoint_name}")
except Exception as e:
    print(f"Deployment failed: {str(e)}")

[02/06/25 13:05:41] INFO     Repacking model artifact                                                  ]8;id=975724;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model.py\model.py]8;;\:]8;id=298337;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model.py#819\819]8;;\
                             (s3://sagemaker-us-east-1-198739141498/deberta-finetuned-model/model.tar.             
                             gz), script artifact (./model_inf), and dependencies ([]) into single                 
                             tar.gz file located at                                                                
                             s3://sagemaker-us-east-1-198739141498/pytorch-inference-2025-02-06-13-05-             
                             41-469/model.tar.gz. This may take some time depending on model size...               

[02/06/25 13:09:21] INFO     Creating model with name: pytorch-inference-2025-02-06-13-09-21-509    ]8;id=423182;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=869018;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[02/06/25 13:09:22] INFO     Creating endpoint-config with name                                     ]8;id=865708;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=62588;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\
                             pytorch-inference-2025-02-06-13-09-22-240                                             

                    INFO     Creating endpoint with name pytorch-inference-2025-02-06-13-09-22-240  ]8;id=834300;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=297619;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

------!Model deployed successfully!
Endpoint Name: pytorch-inference-2025-02-06-13-09-22-240


In [18]:
import boto3

# Initialize SageMaker client
sagemaker_client = boto3.client("sagemaker")

# List all endpoints
endpoints = sagemaker_client.list_endpoints()

# Print all endpoint names
for ep in endpoints["Endpoints"]:
    print("Endpoint Name:", ep["EndpointName"])

Endpoint Name: pytorch-inference-2025-02-06-13-09-22-240


In [ ]:
import boto3
import json

# Initialize the SageMaker runtime client
sagemaker_runtime = boto3.client("sagemaker-runtime")

# Define the SageMaker endpoint
ENDPOINT_NAME = "pytorch-inference-2025-02-06-13-09-22-240"

# Example input text
input_text = "The new 5G technology is improving telecommunications."

# Format the input data as JSON
payload = json.dumps({"inputs": input_text})

# Invoke the SageMaker endpoint
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=ENDPOINT_NAME,  # Your SageMaker endpoint name
    ContentType="application/json",
    Body=payload
)

# Read and parse the response
result = json.loads(response["Body"].read().decode())

print("Model Response:", result)
